Before we begin, let's execute the cell below to display information about the CUDA driver and GPUs running on the server by running the `nvidia-smi` command. To do this, execute the cell block below by giving it focus (clicking on it with your mouse), and hitting Ctrl-Enter, or pressing the play button in the toolbar above. If all goes well, you should see some output returned below the grey cell.

In [ ]:
!nvidia-smi

## Learning objectives
The **goal** of this lab is to:

- Learn how to find bottleneck and performance limiters using Nsight tools
- Learn about ways to to extract more parallelism in loops
- Dig deeper into kernels by analyzing it with Nsight Compute


In this section, we will optimize the parallel [RDF](../serial/rdf_overview.ipynb) application using OpenMP offloading. Before we begin, feel free to have a look at the parallel version of the code and inspect it once again. 

[RDF Parallel Code](../../source_code/openmp/SOLUTION/rdf_offload.f90)

Now, let's compile, and profile it with Nsight Systems first.

In [ ]:
#compile for Tesla GPU
!cd ../../source_code/openmp && nvfortran -mp=gpu -Minfo=mp -o rdf nvtx.f90 SOLUTION/rdf_offload.f90 -L/opt/nvidia/hpc_sdk/Linux_x86_64/21.3/cuda/11.2/lib64 -lnvToolsExt

In [ ]:
#profile and see output of nvptx
!cd ../../source_code/openmp && nsys profile -t nvtx,cuda --stats=true --force-overwrite true -o rdf_offload ./rdf

Let's checkout the profiler's report. [Download the profiler output](../../source_code/openacc/rdf_offload.qdrep) and open it via the GUI. Then, profile the application with Nsight Compute. We use this as the base to compare with the next steps.

In [ ]:
#profile with nsight compute
!cd ../../source_code/openmp && ncu --set full --launch-skip 0 --launch-count 1 -o rdf_offload ./rdf

Let's checkout the profiler's report. [Download the profiler output](../../source_code/openmp/rdf_offload.ncu-rep) and open it via the GUI.

## Further Optimization
Currently both our distributed and workshared parallelism comes from the same loop. In other words, in this example, we used teams and distribute to spread parallelism across the full GPU. As a result, all of the parallelism comes from the outer loop. There are multiple ways to increase parallelism: 

- Moving the `Parallel do` to the inner loop
- Collapsing the loops together

In the following sections, we will explore each of these methods.


### Split Teams Distribute from Parallel Do

In this method, we distribute outer loop to thread teams and workshare inner loop across threads.The example below shows usage of OpenMP target splitting teams distribute from parallel do. The example code contains two levels of OpenMP offloading parallelism in a double-nested loop.

```fortran
!$omp target teams distribute
    do i=1,N
        !$omp parallel do
        do j=1,N
```

Look at the profiler report from the previous section again. From the timeline, have a close look at the the kernel functions. Checkout the theoretical occupancy. As shown in the example screenshot below,`nvkernel_MAIN__F1L98_1_` kernel has the theoretical occupancy of 50%. It clearly shows that the occupancy is a limiting factor. *Occupancy* is a measure of how well the GPU compute resources are being utilized. It is about how much parallelism is running / how much parallelism the hardware could run.

<img src="../images/f_offload_grid.png">


NVIDIA GPUs are comprised of multiple streaming multiprocessors (SMs) where it can manage up to 2048 concurrent threads (not actively running at the same time). Low occupancy shows that there are not enough active threads to fully utilize the computing resources. Higher occupancy implies that the scheduler has more active threads to choose from and hence achieves higher performance. So, what does this mean in OpenACC execution model?


Now, lets start modifying the original code and split the teams distribute. From the top menu, click on *File*, and *Open* `rdf.f90` from the current directory at `Fortran/source_code/openmp` directory. Remember to **SAVE** your code after changes, before running below cells.

In [ ]:
#compile for Tesla GPU
!cd ../../source_code/openmp && nvfortran -mp=gpu -Minfo=mp -o rdf nvtx.f90 SOLUTION/rdf_offload.f90 -L/opt/nvidia/hpc_sdk/Linux_x86_64/21.3/cuda/11.2/lib64 -lnvToolsExt

Inspect the compiler feedback (you should get a similar output as below) you can see from *Line 98* that it separated the teams distribute from parallel do.

<img src="../images/f_openmp_feedback_offload_split.png">

Now, validate the output by running the executable, and then **Profile** your code with Nsight Systems command line `nsys`.

In [ ]:
#Run on Nvidia GPU and check the output
!cd ../../source_code/openmp && ./rdf && cat Pair_entropy.dat

The output should be the following:

```
s2      :    -2.452690945278331     
s2bond  :    -24.37502820694527    
```

In [ ]:
#profile and see output of nvptx
!cd ../../source_code/openmp && nsys profile -t nvtx,cuda --stats=true --force-overwrite true -o rdf_offload_split ./rdf

Let's checkout the profiler's report. [Download the profiler output](../../source_code/openmp/rdf_offload_split.qdrep) and open it via the GUI. Compare the execution time for the `Pair Calculation` from the NVTX row with the previous section and see how much the performance was improved (you should be getting similar screenshots as below where 1) is the base version and 2) is the current version after splitting the "Teams Distribute" from "Parallel Do".  The "Pair Calculation" reduced from 61.2 ms to 45.3 ms.

<img src="../images/f_offload_compare_nvtx.png">

Let's dig deeper and profile the kernel with Nsight Compute and compare it with the base version.

In [ ]:
#profile with nsight compute
!cd ../../source_code/openmp && ncu --set full --launch-skip 0 --launch-count 1 -o rdf_offload_split ./rdf

Let's checkout Nsight Compute report. [Download the profiler output](../../source_code/openmp/rdf_offload_split.ncu-rep) and open it via the GUI. Have a look at the example expected profiler report below:

<img src="../images/f_openmp_offload_split_cmp.png">

When compared to the base version, we achieved 124% improvement in SM utilization and 709% utilization in memory. 

Feel free to checkout the [solution](../../source_code/openmp/SOLUTION/rdf_offload_split.f90) to help you understand better.

### `num_teams(n)` and `thread_limit`

When using the `distribute parallel do` construct, the *for loop* is distributed across all threads for all teams of the current teams region. For example, if there are 10 teams, and each team consists of 256 threads, the loop will be distributed across 2560 threads. You can explicitly specify the number of threads to be created in the team using the `thread_limit(m)`. Moreover, you can specify the maximum number of teams created by using `num_teams(n)` (the actual number of teams may be smalled than this number). You can add these attributes to the `teams` construct. Please note that you cannot add `num_teams` to the `parallel` construct clause. For more information, please read the guide at https://docs.nvidia.com/hpc-sdk/compilers/hpc-compilers-user-guide/#openmp-subset .


```fortran
!$omp target teams distribute num_teams(65535)
do i=1,N
    !$omp parallel do thread_limit(128)
    do j=1,N
        < loop code >
```
In the previous section, the grid size was too small to keep the hardware busy. Take a look at the *Launch Statistics*, the grid size is 80. 

<img src="../images/f_openmp_offload_split_grid.png">

Now, lets start modifying the code again and add `thread_limit(n)` to the `teams` construct. From the top menu, click on *File*, and *Open* `rdf.f90` from the current directory at `Fortran/source_code/openmp` directory. Remember to **SAVE** your code after changes, before running below cells.

**NOTE:** Try increasing the grid size to 65525 (*hint:* use `num_teams(65535)`). Once done, compile and profile the code and compare it with the previous version. Are there any differences? What does the profiler show?

In [ ]:
#compile for Tesla GPU
!cd ../../source_code/openmp && nvfortran -mp=gpu -Minfo=mp -o rdf nvtx.f90 SOLUTION/rdf_offload.f90 -L/opt/nvidia/hpc_sdk/Linux_x86_64/21.3/cuda/11.2/lib64 -lnvToolsExt

Now, validate the output by running the executable, and then **Profile** your code with Nsight Systems command line `nsys`.

In [ ]:
#Run on Nvidia GPU and check the output
!cd ../../source_code/openmp && ./rdf && cat Pair_entropy.dat

The output should be the following:

```
s2      :    -2.452690945278331     
s2bond  :    -24.37502820694527    
```

In [ ]:
#profile and see output of nvptx
!cd ../../source_code/openmp && nsys profile -t nvtx,cuda --stats=true --force-overwrite true -o rdf_offload_split_num ./rdf

Let's checkout the profiler's report. [Download the profiler output](../../source_code/openmp/rdf_offload_split_num.qdrep) and open it via the GUI. Let's dig deeper and profile the application with Nsight Compute and compare it with the baseline.

In [ ]:
#profile with nsight compute
!cd ../../source_code/openmp && ncu --set full --launch-skip 0 --launch-count 1 -o rdf_offload_split_num ./rdf

Let's checkout the Nsight Compute report. [Download the profiler output](../../source_code/openmp/rdf_offload_split_num.ncu-rep) and open it via the GUI. 

<img src="../images/f_openmp_offload_split_cmp2.png">

Comparing to the previous version, we achieved 72.39% less SM utilization. Now, let's compare the two optimization approaches with the base version (using target offload). As seen in the roofline chart below, the application is now in the *memory bound* region which means either it is limited by memory bandwidth or latency. Memory is heavily itilized than compute so it appears to be limited by memory bandwidth.

<img src="../images/f_openmp_offload_roofline.png">

The kernel exhibits low compute throughput and memory bandwidth utilization relative to the peak performance of the device. Let's have a look at the *Warp State Statistics* section for potential reasons.

<img src="../images/f_openmp_warp_cmp.png">

Comparing the two optimizations, warp is stalled waiting for the L1 instruction queue for local and global (LG) memory operations to be not full. The instruction queue is full. 

It is very clear that we did not improve anything through previous approaches. In the following sections, we look at the `collapse` clause to improve the performance.

Feel free to checkout the example [solution](../../source_code/openmp/SOLUTION/rdf_offload_offload_split_num.f90) to help you understand better.

## Collapse clause

Specifying the `collapse(n)` clause takes the next `n` tightly-nested loops, folds them into one, and applies the OpenMP directives to the new loop. Collapsing loops means that two loops of trip counts N and M respectively will be automatically turned into a single loop with a trip count of N times M. By collapsing two or more parallel loops into a single loop the compiler has an increased amount of parallelism to use when mapping the code to the device. On highly parallel architectures, such as GPUs, this will give us more parallelism to distribute and better performance.

Try using the collapse clause and observe any performance difference. How much this optimization will speed-up the code will vary according to the application and the target accelerator, but it is not uncommon to see large speed-ups by using collapse on loop nests.

In the example below, we collapse the two loops before applying both teams and thread parallelism to both.

```fortran
!$omp target teams distribute parallel do collapse(2)
do i=1,N
    do j=1,N
```

Now, lets start modifying the original code before splitting the `teams distribute` from `parallel do` and add the collapse clause. From the top menu, click on *File*, and *Open* `rdf.f90` from the current directory at `Fortran/source_code/openmp` directory. Remember to **SAVE** your code after changes, before running below cells.

In [ ]:
#compile for Tesla GPU
!cd ../../source_code/openmp && nvfortran -mp=gpu -Minfo=mp -o rdf nvtx.f90 rdf.f90 -L/opt/nvidia/hpc_sdk/Linux_x86_64/21.3/cuda/11.2/lib64 -lnvToolsExt

Now, validate the output by running the executable, and then **Profile** your code with Nsight Systems command line `nsys`.

In [ ]:
#Run on Nvidia GPU and check the output
!cd ../../source_code/openmp && ./rdf && cat Pair_entropy.dat

The output should be the following:

```
s2      :    -2.452690945278331     
s2bond  :    -24.37502820694527    
```

In [ ]:
#profile and see output of nvptx
!cd ../../source_code/openmp && nsys profile -t nvtx,cuda --stats=true --force-overwrite true -o rdf_collapse ./rdf

Let's checkout the profiler's report. [Download the profiler output](../../source_code/openmp/rdf_collapse.qdrep) and open it via the GUI. Have a look at the example expected profiler report below:

<img src="../images/f_openmp_gpu_collapse.png">

Compare the execution time for the `Pair Calculation` from the NVTX row (annotated in Red rectangle in the example screenshot) with the previous section. It is clear the using collapse clause improved the performance by extracting more parallelism. Please note when you compare the two methods explored here, one may give better results than the other.

Let's dig deeper and profile the kernel with Nsight Compute and compare it with the base version.

In [ ]:
#profile with nsight compute
!cd ../../source_code/openmp && ncu --set full --launch-skip 0 --launch-count 1 -o rdf_collapse ./rdf

Let's checkout the Nsight Compute report. [Download the profiler output](../../source_code/openmp/rdf_collapse.ncu-rep) and open it via the GUI. Have a look at the example expected profiler report below (yellow is the base version, blue is the current):

<img src="../images/f_openmp_offload_collapse.png">

When compared to the base version (using baseline feature on Nsight Compute), we achieved 262% improvement in SM utilization and 801% utilization in memory. We can also see the improvement and better utilization in the roofline analysis (see example screenshot below).The ideal situation would be getting closer to the rooflines (up).

<img src="../images/f_openmp_collapse_baseline.png">

Dots with the blue outline is the optimized version using `collapse` clause and the ones with the orange outline is the base version.

Lets checkout the *Occupancy* section. Occupancy is the ratio of the number of active warps per multiprocessor to the maximum number of possible active warps. As seen in the below screenshot, the theoretical occupancy is 43.7% and the achieved occupancy is 41.3%. The plot showing the impact of varying register count per thread indicates that by reducing the number of registers per thread, we can increase the warp occupancy. 

<img src="../images/f_openmp_offload_occupancy.png">

We can limit the register count at compile time by adding `-gpu=maxregcount:32` flag. Based on the plot, let reduce the number of registers per thread to 32 to see if it improves the overall performance. 

In [ ]:
#compile for Tesla GPU
!cd ../../source_code/openmp && nvfortran -mp=gpu -gpu=maxregcount:32 -Minfo=mp -o rdf nvtx.f90 SOLUTION/rdf_offload_collapse.f90 -L/opt/nvidia/hpc_sdk/Linux_x86_64/21.3/cuda/11.2/lib64 -lnvToolsExt

Now, validate the output by running the executable, and then **Profile** your code with Nsight Systems command line `nsys`.

In [ ]:
#Run on Nvidia GPU and check the output
!cd ../../source_code/openmp && ./rdf && cat Pair_entropy.dat

The output should be the following:

```
s2      :    -2.452690945278331     
s2bond  :    -24.37502820694527    
```

In [ ]:
#profile and see output of nvptx
!cd ../../source_code/openmp && nsys profile -t nvtx,cuda --stats=true --force-overwrite true -o rdf_offload_collapse_regcount ./rdf

Let's checkout the profiler's report. [Download the profiler output](../../source_code/openmp/rdf_collapse_regcount.qdrep) and open it via the GUI. Let's dig deeper and profile the application with Nsight Compute and compare it with the baseline.

In [ ]:
#profile with nsight compute
!cd ../../source_code/openmp && ncu --set full --launch-skip 0 --launch-count 1 -o rdf_collapse_regcount ./rdf

Let's checkout the Nsight Compute report. [Download the profiler output](../../source_code/openmp/rdf_collapse_regcount.ncu-rep) and open it via the GUI. Have a look at the example expected profiler report below (pink is the previous version using `collapse` clause, blue is the current after limiting the register count):

<img src="../images/f_openmp_collapse_reg.png">

We achieved 600% utilization in memory and 5% in SM. The current version takes 102% longer to execute. Roofline chart shows the application is now in memory bound region comparing to before that was in compute bound region (dots with the pink outline is the base version using `collapse` clause and the ones with the blue outline is the current version).

<img src="../images/f_openmp_collapse_reg_roofline.png">

Lets checkout the *Occupancy* section. As seen in the below screenshot, the theoretical occupancy is 100% and the achieved occupancy is 94.6%. Higher occupancy does not always result in higher performance, however, low occupancy always reduces the ability to hide latencies, resulting in overall performance degradation.


<img src="../images/f_openmp_collapse_reg_occupancy.png">

Let's have a look at the memory chart in the *memory workload analysis* section. Links between local memory and L1/TEX cache show an increase in the number of requests generated due to local memory load/store instructions. These are the result of registers spilling to local memory that happened after reducing the number of registers per thread to increase the occupancy. Once can also see the registers spilling by using `-Mcuda=ptxinfo` option at the compile time. Below is an example output showing the number of bytes spills to local memory. To read more about PTX, checkout the guide at https://docs.nvidia.com/cuda/parallel-thread-execution/index.html#ptx-machine-model .

```
ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function 'nvkernel_MAIN__F1L98_1_' for 'sm_70'
ptxas info    : Function properties for nvkernel_MAIN__F1L98_1_
    88 bytes stack frame, 128 bytes spill stores, 120 bytes spill loads
ptxas info    : Used 32 registers, 720 bytes cmem[0]
```

It is very clear that reducing the number of registers, had impact on memory and it hurt the performance by increasing the instruction count as well as memory traffic.

<img src="../images/f_openmp_collapse_reg_memory.png">

Feel free to checkout the example [solution](../../source_code/openmp/SOLUTION/rdf_offload_collapse.f90) to help you understand better.

## Post-Lab Summary

If you would like to download this lab for later viewing, it is recommend you go to your browsers File menu (not the Jupyter notebook file menu) and save the complete web page.  This will ensure the images are copied down as well. You can also execute the following cell block to create a zip-file of the files you've been working on, and download it with the link below.

In [ ]:
%%bash
cd ..
rm -f nways_files.zip
zip -r nways_files.zip *

**After** executing the above zip command, you should be able to download the zip file [here](../nways_files.zip). Let us now go back to parallelizing our code using other approaches.

**IMPORTANT**: Please click on **HOME** to go back to the main notebook for *N ways of GPU programming for MD* code.

-----

# <p style="text-align:center;border:3px; border-style:solid; border-color:#FF0000  ; padding: 1em"> <a href=../../../nways_MD_start.ipynb>HOME</a></p>

-----


# Links and Resources
[OpenMP Programming Model](https://computing.llnl.gov/tutorials/openMP/)

[OpenMP Target Directive](https://www.openmp.org/wp-content/uploads/openmp-examples-4.5.0.pdf)

[NVIDIA Nsight System](https://docs.nvidia.com/nsight-systems/)

[NVIDIA Nsight Compute](https://developer.nvidia.com/nsight-compute)

**NOTE**: To be able to see the Nsight System profiler output, please download Nsight System latest version from [here](https://developer.nvidia.com/nsight-systems).

Don't forget to check out additional [OpenACC Resources](https://www.openacc.org/resources) and join our [OpenACC Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

--- 

## Licensing 

This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).